In [5]:
import torch

# General Python Packages
import os, time, numbers, math

# Torch Packages
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torch.optim import lr_scheduler, SGD
from torch.autograd import Variable
from torch import nn
from torch.nn import DataParallel
from torch.nn import Module

# General Analytics Packages
import pandas as pd
import numpy as np

# Visualization / Image Packages
import matplotlib.pyplot as plt
from PIL import Image

# Randomization Functions
from random import random as randuni

/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/font_manager.py:279: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


### Plotting Functions for Model Evaluations

In [1]:
def plot_auc_bar(in_evaluator):
    auc_out = in_evaluator.m_auc.value()
    plt.bar(range(14), [v for v, _, _ in auc_out])

In [3]:
def plot_auc_curves(in_evaluator):
    index_to_label = {idx: val for val, idx in img_data_train.label_to_index.items()}
    auc_out = in_evaluator.m_auc.value()
    
    plt.figure(figsize=(10,8))

    for idx, (auc, tpr, fpr) in enumerate(auc_out):
        disease = index_to_label[idx]
        plt.plot(fpr, tpr, label="{0} (AUC: {1:0.3f})".format(disease, auc))

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve by Disease')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
def plot_compare_sums(in_evaluator, bar_width = 0.4):
    sums_out = in_evaluator.m_total_sums.get_results()
    
    act = sums_out['act'].data.numpy()
    pred = sums_out['pred'].data.numpy()
    
    indx1 = range(14)
    indx2 = [i+bar_width for i in indx1]

    plt.bar(indx1, act,  width=bar_width, label="Actual")
    plt.bar(indx2, pred, width=bar_width, label="Predicted")
    
    plt.xticks(indx1, img_data_train.labels, rotation=90)
    plt.legend()

# Loading & Analysis of Model Results

In [5]:
epoch_evals = {}

In [4]:
test_load = torch.load('/user/rn18ff_model_9.tar')

In [ ]:
epoch = learning_scheduler.last_epoch

epoch_evals[epoch] = {}

epoch_evals[epoch]['val'] = ModelEvaluator(model_ft, dataloaders['val'])
#epoch_evals[epoch]['train'] = ModelEvaluator(model_ft, dataloaders['train'])

epoch_evals[epoch]['val'].run()
#epoch_evals[epoch]['train'].run()

In [ ]:
plot_auc_curves(epoch_evals[epoch]['val'])

In [ ]:
plt.figure(figsize=(15,6))
plt.subplot(1,2,1)
plt.title("Training Sums")
#plot_compare_sums(epoch_evals[epoch]['train'])

plt.subplot(1,2,2)
plt.title("Validation Sums")
plot_compare_sums(epoch_evals[epoch]['val'])

In [ ]:
load_opt = test_load['optimizer']
load_sched = test_load['scheduler']
load_state = test_load['state']
# Not used: 'epoch', 'val_error'

In [ ]:
load_state

In [ ]:
model2 = models.resnet18(pretrained=True)
for param in model2.parameters():
    param.requires_grad = False

# Replace FC layer
model2.fc = nn.Linear(model2.fc.in_features, len(img_data_train.labels))

model2_c = DataParallel(model2).cuda()

In [ ]:
model2_c.load_state_dict(load_state)

In [ ]:
model2_c.forward(Variable(img_data_train[0][0].unsqueeze(0).cuda())).sigmoid()

In [ ]:
def single_model_run(model, dataset):
    dataset
    for data in self.dataset:
        inputs, actuals = data

        inputs = Variable(inputs.cuda(), volatile=True)
        actuals = Variable(actuals.cuda(), volatile=True)

        outputs = self.model(inputs)
        preds = outputs.sigmoid()

        self.m_total_sums.update(preds, actuals)
        self.m_auc.add(preds, actuals)

In [ ]:
out_model_30.train(mode=False)

obs_counter = 0
total_pred = Variable(torch.FloatTensor(torch.zeros(14)))
total_act = Variable(torch.FloatTensor(torch.zeros(14)))

conf_a = {}
conf_b = {}
conf_c = {}
conf_d = {}
for i in range(1,10):
    conf_a[i] = Variable(torch.FloatTensor(torch.zeros(14)))
    conf_b[i] = Variable(torch.FloatTensor(torch.zeros(14)))
    conf_c[i] = Variable(torch.FloatTensor(torch.zeros(14)))
    conf_d[i] = Variable(torch.FloatTensor(torch.zeros(14)))

for data in dataloaders['val']:
    print("STARTING ITERATION...")
    inputs, labels = data
    print("PROCESSING FIRST {} OBSERVATIONS".format(len(inputs)))

    inputs = Variable(inputs.cuda())
    labels = Variable(labels.cuda())

    outputs = out_model_30(inputs).sigmoid()
    
    total_act += labels.sum(0).cpu()
    total_pred += outputs.sum(0).cpu()

    # Store statistics (convert from autograd.Variable to float/int)
    for i in range(1,10):
        t = i/10
        conf_a[i] += ((outputs.sigmoid()>t) == (labels>0.5)).sum(0).cpu().float()
        conf_b[i] += ((outputs.sigmoid()<t) == (labels>0.5)).sum(0).cpu().float()
        conf_c[i] += ((outputs.sigmoid()>t) == (labels<0.5)).sum(0).cpu().float()
        conf_d[i] += ((outputs.sigmoid()<t) == (labels<0.5)).sum(0).cpu().float()

    obs_counter += len(inputs)

In [ ]:
comparison = Variable(torch.FloatTensor(9, 14))
for i in range(9):
    comparison[0] = conf_a[1] / obs_counter
print(comparison.int())

In [ ]:
def single_model_run(model, dataset):
    dataset
    for data in self.dataset:
        inputs, actuals = data

        inputs = Variable(inputs.cuda(), volatile=True)
        actuals = Variable(actuals.cuda(), volatile=True)

        outputs = self.model(inputs)
        preds = outputs.sigmoid()

        self.m_total_sums.update(preds, actuals)
        self.m_auc.add(preds, actuals)